In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

In [3]:
import urllib.parse

In [4]:
def extract_book_info(card_id):
    html = 'https://www.chitai-gorod.ru/catalog/book/{id_}'.format(id_=card_id)
    card_html = requests.get(html).text
    soup = BeautifulSoup(card_html, 'html.parser')
    ret = {
    'ID карточки' : card_id,
    'Название' : soup.find('h1', class_='product__title js-analytic-product-title').text,
    'Автор' : soup.find('a', class_='link product__author', href=True).text.strip(),
    'Рейтинг' : soup.find('div', class_='rating').text.strip()[0],
    'Голоса' : soup.find('div', class_='rating').text.strip()[3:-1]
    }
    tmp_f = soup.find('div', class_='price')
    if tmp_f is not None:
        ret['Цена'] = tmp_f.text.strip()[:-2]
    for i in soup.find_all('div', class_='product__props'):
        for j in i.find_all('div', class_='product-prop'):
            field = j.find('div', class_='product-prop__title').text.strip()
            value = j.find('div', class_='product-prop__value').text.strip()
            ret[field] = value
    tmp_f = soup.find('div', class_='product__image').find('img')
    if tmp_f is not None:
        ret['Обложка'] = tmp_f['src']
    return ret

In [5]:
from selenium import webdriver

In [6]:
from time import sleep

In [7]:
import random

def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

In [8]:
def get_ids(author, driver):
    global ids
    html = 'https://www.chitai-gorod.ru/search/result.php?q={author}&type=author'.format(\
        author=urllib.parse.quote(author, encoding='windows-1251'))
    driver.get(html)
    random_sleep()
    #Здесь я не придумал более здравого способа отобразить сразу все книги
    driver.execute_script("window.scrollTo(0, 3000)") 
    random_sleep()
    driver.execute_script("window.scrollTo(0, 6000)") 
    random_sleep()
    driver.execute_script("window.scrollTo(0, 10000)")
    random_sleep()
    button = driver.find_element_by_class_name('catalog__btn_show-more')
    button.click()
    random_sleep()
    driver.execute_script("window.scrollTo(0, 16000)")
    random_sleep()
    driver.execute_script("window.scrollTo(0, 21000)")
    random_sleep()
    driver.execute_script("window.scrollTo(0, 29000)")
    #Конец костыля здесь
    items = driver.find_elements_by_class_name("product-card")
    for item in items:
        ids.append(item.get_attribute('data-product'))
    random_sleep()
    return

In [9]:
ids = list()
with webdriver.Firefox() as driver:
    get_ids("Макс Фрай", driver)
    get_ids("Эрин Хантер", driver)
    get_ids("Дмитрий Емец", driver)

In [10]:
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool

In [11]:
t = ThreadPool(16)
result = list(t.map(extract_book_info, ids))
df = pd.DataFrame(result)
t.close()
t.join()
df.sort_values(by='ID карточки', inplace=True)

In [12]:
df.head()

,ID карточки,ID товара:,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Обложка,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена
108,1000078,2606052,9785179826767,"Фрай М., Горалик Л.",18+,2017,3,АСТ,256,Книга одиночеств,https://img-gorod.ru/upload/iblock/55f/55fb1a4...,NaN,NaN,4,Миры Макса Фрая,Твердая бумажная,3000,20.5 x 13 x 1.4,NaN,332
42,1004167,2608544,9785171053802,Фрай М.,18+,2017,4,АСТ,288,Новая чайная книга,https://img-gorod.ru/upload/iblock/f22/f22f1a0...,NaN,NaN,5,Миры Макса Фрая,Твердая бумажная,7000,20.6 x 13.2 x 2.3,NaN,332
141,1004240,2608971,9785001111900,Хантер Э.,6+,2017,1,Абрис,350,Расколотый прайд,https://img-gorod.ru/upload/iblock/b32/b32d2f0...,Голосовская А.Ю.,NaN,5,Земля Отважных,Твердая бумажная,7000,20.6 x 13 x 1.7,Насыров Л.Х.,285
140,1008090,2612352,9785001111733,Хантер Э.,6+,2017,3,Абрис,350,Нерассказанные истории. Ярость когтя,https://img-gorod.ru/upload/iblock/938/93828c3...,Вероника Максимова,NaN,5,Коты-воители,Твердая бумажная,6000,20.7 x 13.2 x 2,NaN,304
41,1008621,2612809,9785171053819,Фрай М.,18+,2018,7,АСТ,320,Новая кофейная книга,https://img-gorod.ru/upload/iblock/5cd/5cd20b7...,NaN,NaN,4,Миры Макса Фрая,Твердая бумажная,8000,20.7 x 13.2 x 2.6,NaN,332
